# Import data and libraries

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')
df.sample(5)

,age,gender,fever,cough,city,has_covid
97,20,Female,101.0,Mild,Bangalore,No
14,51,Male,104.0,Mild,Bangalore,No
93,27,Male,100.0,Mild,Kolkata,Yes
21,73,Male,98.0,Mild,Bangalore,Yes
0,60,Male,103.0,Mild,Kolkata,No


In [4]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

# Train test split

In [7]:
X = df.drop(columns=['has_covid'])
y = df['has_covid']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
54,60,Female,99.0,Mild,Mumbai
76,80,Male,100.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata
83,17,Female,104.0,Mild,Kolkata
...,...,...,...,...,...
2,42,Male,101.0,Mild,Delhi
98,5,Female,98.0,Strong,Mumbai
96,51,Female,101.0,Strong,Kolkata
86,25,Male,104.0,Mild,Bangalore


# 1. Normal Life (W/o column Transformer)

##### Simple imputer -> Fever

In [13]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

##### Ordinal encoding  -> Cough

In [14]:
oe = OrdinalEncoder()

X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

##### One Hot encoding  -> Gender and city

In [16]:
ohe = OrdinalEncoder()

X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

X_train_gender_city.shape

(80, 2)

##### Extracting age

In [17]:
X_train.columns

Index(['age', 'gender', 'fever', 'cough', 'city'], dtype='object')

In [20]:
X_train_age = X_train['age'].values
X_test_age = X_test['age'].values

X_train_age.shape

(80,)

In [ ]:
# due this is not is the same size so we reshape for the concatenate
X_train_age = X_train_age.reshape(-1, 1)
X_test_age = X_test_age.reshape(-1, 1)

##### Now Concatenate Everything

In [28]:
print(X_test_age.shape)
print(X_test_cough.shape)
print(X_test_fever.shape)
print(X_test_gender_city.shape)

(20, 1)
(20, 1)
(20, 1)
(20, 2)


In [29]:
X_train_transformed = np.concatenate((
    X_train_age,
    X_train_cough,
    X_train_fever,
    X_train_gender_city
), axis=1)

X_test_transformed = np.concatenate((
    X_test_age,
    X_test_cough,
    X_test_fever,
    X_test_gender_city
), axis=1)

In [31]:
np.round(X_train_transformed)

array([[ 81.,   0., 101.,   0.,   3.],
       [ 60.,   0.,  99.,   0.,   3.],
       [ 80.,   0., 100.,   1.,   0.],
       [ 82.,   1., 102.,   0.,   2.],
       [ 17.,   0., 104.,   0.,   2.],
       [ 19.,   0., 100.,   0.,   2.],
       [ 23.,   0., 101.,   1.,   3.],
       [ 64.,   0.,  98.,   0.,   0.],
       [ 11.,   1., 100.,   0.,   2.],
       [ 72.,   0.,  99.,   1.,   0.],
       [ 24.,   0.,  98.,   1.,   2.],
       [ 16.,   0., 103.,   0.,   0.],
       [  5.,   0., 102.,   1.,   2.],
       [ 73.,   0., 103.,   0.,   1.],
       [ 10.,   1.,  98.,   0.,   2.],
       [ 65.,   0.,  99.,   1.,   1.],
       [ 47.,   1., 101.,   1.,   0.],
       [ 14.,   0.,  99.,   0.,   3.],
       [ 20.,   0., 101.,   0.,   0.],
       [ 27.,   0., 100.,   1.,   2.],
       [ 70.,   1., 103.,   1.,   2.],
       [ 20.,   1., 102.,   1.,   1.],
       [ 50.,   0., 103.,   0.,   2.],
       [ 38.,   0., 101.,   0.,   0.],
       [ 71.,   1.,  98.,   0.,   2.],
       [ 60.,   0., 103.,

In [32]:
X_train_transformed.shape

(80, 5)

# 2. Mentors Life (W/ Columns Transformer)

In [43]:
from sklearn.compose import ColumnTransformer

In [44]:
transformer = ColumnTransformer(transformers=[
    (),
    (),
    (),
], remainder='passthrough')

In [48]:
transformer = ColumnTransformer(transformers=[
    ('trf1', SimpleImputer(), ['fever']),
    ('trf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('trf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city']),
], remainder='passthrough')

transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('trf1', SimpleImputer(), ['fever']),
                                ('trf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('trf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [50]:
transformer.fit_transform(X_train)

array([[101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  60.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  80.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [101.01408451,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.    

In [51]:
transformer.fit_transform(X_train).shape

(80, 7)

In [52]:
test_transformer = transformer.fit_transform(X_test)
test_transformer

array([[ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  82.        ],
       [ 98.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  83.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  80.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.    

In [53]:
test_transformer.shape

(20, 7)